In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd drive/MyDrive/MiniProject/Project/Tabu Search

/content/drive/.shortcut-targets-by-id/1R5lktF0Aag9Aa-GqEI_uQJTKOkHYAbQG/MiniProject/Project/TabuSearch


In [ ]:
import numpy as np
import copy
import random
import time
import os

In [ ]:
def loadData(Type, Name):
    global N, m, M, d, s, e, date
    f = open("Data/"+Type+"/"+Name+".txt", "r")

    param = f.readline().split()
    N = int(param[0])
    m = int(param[1])
    M = int(param[2])

    d = []
    s = []
    e = []
    for i in range(N):
        line = f.readline().split()
        d.append(int(line[0]))
        s.append(int(line[1]))
        e.append(int(line[2]))

    f.close()

    date = np.max(e)+1

    return N, m, M, d, s, e, date

In [ ]:
def initialSolution(SEED):
    random.seed(SEED)
    sol = np.array(s)
    amount = np.zeros(date)
    global flag
    flag = [0]*date

    # cánh đồng nào thu nhiều thì ưu tiên chọn trước
    order = np.argsort(-np.array(d))
    for i in order:
        # sol[i] = random.randint(s[i],e[i])
        temp1 = [j for j in range(s[i], e[i]+1) if flag[j] == 0]
        temp2 = [j for j in range(s[i], e[i]+1) if flag[j] == 1]
        temp3 = [j for j in range(s[i], e[i]+1)
                 if (flag[j] == 2 and amount[j] + d[i] <= M)]
        if (len(temp1) != 0):
            day = random.choice(temp1)
        elif (len(temp2) != 0):
            day = random.choice(temp2)
        elif (len(temp3) != 0):
            day = random.choice(temp3)
        else:
            day = random.randint(s[i], e[i])
        # chọn ngày cho cánh đồng i
        sol[i] = day
        amount[day] += d[i]
        if (amount[day] > M):
            flag[day] = 3  # đã đến giới hạn
        elif (amount[day] >= m):
            flag[day] = 2  # vừa đủ
        elif (amount[day] > 0):
            flag[day] = 1  # chưa đủ
    # điều chỉnh thêm
    for i in range(4):
        sol = reset(sol, SEED+i)

    return sol

In [ ]:
def reset(sol, SEED):
    random.seed(SEED)
    global flag
    flag = [0]*date
    amount = np.zeros(date)
    day = [[] for _ in range(N)]
    for i in range(N):
        amount[sol[i]] += d[i]
        day[sol[i]].append(i)

    for i in range(date):
        if amount[i] > M:
            flag[i] = 3
        elif amount[i] >= m:
            flag[i] = 2
        elif amount[i] > 0:
            flag[i] = 1

    newSolve = np.array(sol)
    randomOrder = np.arange(date)

    random.shuffle(randomOrder)

    # nếu thu hoạch thừa thì dồn sang ngày khác thiếu
    for i in randomOrder:
        for j in day[i]:  # cánh đồng j thu hoạch ngày i
            if (flag[i] != 3):
                break
            temp1 = [k for k in range(s[j], e[j]+1) if flag[k] <= 1]
            temp2 = [k for k in range(
                s[j], e[j]+1) if (flag[k] == 2 and amount[k] + d[j] <= M)]
            if (len(temp1) != 0):
                k = random.choice(temp1)
            elif (len(temp2) != 0):
                k = random.choice(temp2)
            else:
                continue  # không được thì chuyển sang cánh đồng khác cũng thu vào ngày i

            # cánh đồng j thu hoạch ngày i chuyển sang ngày k
            day[i].remove(j)
            amount[i] -= d[j]
            if (amount[i] < m):
                flag[i] = 1
            elif (amount[i] <= M):
                flag[i] = 2

            newSolve[j] = k
            day[k].append(j)
            amount[k] += d[j]
            if (amount[k] > M):
                flag[k] = 3
            elif (amount[k] >= m):
                flag[k] = 2
            else:
                flag[k] = 1

    random.shuffle(randomOrder)
    # nếu thu hoạch chưa đủ thì dồn sang ngày khác cũng thiếu phía sau
    for i in range(date):
        for j in day[i]:  # cánh đồng j thu hoạch ngày i
            if (flag[i] != 1):
                break

            temp1 = [k for k in range(i+1, e[j]+1) if flag[k] == 1]
            temp2 = [k for k in range(
                s[j], e[j]+1) if (flag[k] == 2 and amount[k] + d[j] <= M)]
            if (len(temp1) != 0):
                k = random.choice(temp1)
            elif (len(temp2) != 0):
                k = random.choice(temp2)
            else:
                continue

            day[i].remove(j)
            amount[i] -= d[j]
            if (amount[i] == 0):
                flag[i] = 0

            day[k].append(j)
            newSolve[j] = k
            amount[k] += d[j]
            if (amount[k] > M):
                flag[k] = 3
            elif (amount[k] >= m):
                flag[k] = 2
            else:
                flag[k] = 1

    random.shuffle(randomOrder)
    # nếu thu hoạch có đủ thì dồn sang ngày khác thiếu hoặc ít hơn
    for i in randomOrder:
        for j in day[i]:  # cánh đồng j thu hoạch ngày i
            if (not (flag[i] == 2 and amount[i] - d[j] >= m)):
                continue

            temp1 = [k for k in range(s[j], e[j]+1) if flag[k] == 1]
            temp2 = [k for k in range(
                s[j], e[j]+1) if (flag[k] == 2 and amount[k]+d[j] < amount[i])]
            if (len(temp1) != 0):
                k = random.choice(temp1)
            elif (len(temp2) != 0):
                k = random.choice(temp2)
            else:
                continue

            day[i].remove(j)
            amount[i] -= d[j]
            # if (amount[i]>=m):
            #     flag[i]=2
            # elif (amount[i]<m):
            #     flag[i]=1

            day[k].append(j)
            newSolve[j] = k
            amount[k] += d[j]
            if (amount[k] >= M):
                flag[k] = 3
            elif (amount[k] >= m):
                flag[k] = 2
            else:
                flag[k] = 1

    # print(amount)
    return newSolve

In [ ]:
def getCost(sol):
    amount = np.zeros(date)
    violation = 0
    for i in range(N):
        amount[sol[i]] += d[i]
    for i in amount:
        violation += max(i-M, 0)+ max(0, min(i, m-i))
    cost = np.max(amount)- np.min(amount)
    return int(cost + 1000 * violation)

In [ ]:
def show(sol):
    amount = np.zeros(date)
    day = [[] for i in range(date)]
    for i in range(N):
        amount[sol[i]] += d[i]
        day[sol[i]].append(i)

    print("Objective value: {}".format(getCost(sol)))

    for i in range(date):
        print("Day {} : {}".format(i, day[i]))
        print("----sum: {}".format(amount[i]))

# Tabu Search


In [ ]:
def search_next_solution(sol, SEED):
    random.seed(SEED)
    select_field = -1
    select_day = -1
    new_search_objective = 1e9
    global flag
    flag = [0]*date

    amount = np.zeros(date)
    violation = 0
    for i in range(N):
        amount[sol[i]] += d[i]
    for i in range(date):
        violation += max(amount[i]-M, 0) + max(0, min(amount[i], m-amount[i]))

        if (amount[i] > M):
            flag[i] = 3  # đã đến giới hạn
        elif (amount[i] >= m):
            flag[i] = 2
        elif (amount[i] > 0):
            flag[i] = 1
    # cost = np.max(amount) - np.min(amount) + 1000*violation

    randomOrder = np.arange(N)
    random.shuffle(randomOrder)
    for i in randomOrder:
        if tabuList[i] > 0:
            continue
        for j in range(s[i], e[i]+1):
            if (j == sol[i] or flag[j] == 0 or flag[j] == 3):
                continue
            max(i-M, 0)+max(0, min(i, m-i))
            oldViolation = max(amount[sol[i]]-M, 0)+max(0, min(amount[sol[i]], m-amount[sol[i]])) + max(amount[j]-M, 0)+max(0, min(amount[j], m-amount[j]))
            amount[sol[i]] -= d[i]
            amount[j] += d[i]
            newViolation = max(amount[sol[i]]-M, 0)+max(0, min(amount[sol[i]], m-amount[sol[i]])) + max(amount[j]-M, 0)+max(0, min(amount[j], m-amount[j]))
            newSolObjective = np.max(amount) - np.min(amount) + 1000*(violation - oldViolation + newViolation)
            # restore
            amount[sol[i]] += d[i]
            amount[j] -= d[i]

            if newSolObjective < new_search_objective:
                new_search_objective = newSolObjective
                select_field = i
                select_day = j

    return select_field, select_day, new_search_objective

In [ ]:
def TabuSearch(outputPath, Name, SEED):
    solve = initialSolution(SEED)
    bestSolve = np.array(solve)

    global tabuList
    itmax = 300
    it = 0
    tabuList = np.zeros(N)
    tbl = 7
    tbl_min = 5
    tbl_max = 9
    stable = 0
    stable_limit = 10
    restart_freq = 25

    f = open(outputPath, "w")
    f.write("Name: "+Name+"\n")

    # show(solve)
    # print("Gen{}: {} {}".format(it,getCost(solve),getCost(bestSolve)))
    f.write("Gen{}: {} {}\n".format(it, getCost(solve), getCost(bestSolve)))

    while (it < itmax):
        it += 1

        # Move
        oldSolve = np.array(solve)
        old_search_objective = getCost(solve)
        field, day, new_search_objective = search_next_solution(solve, SEED+it)
        solve[field] = day
        search_objective = new_search_objective

        # Update tabu
        for i in range(N):
            if tabuList[i] > 0:
                tabuList[i] -= 1
        tabuList[field] = tbl

        if new_search_objective < old_search_objective:
            if tbl > tbl_min:
                tbl -= 1
        else:
            if tbl < tbl_max:
                tbl += 1

        # Update last improved solution
        if new_search_objective < old_search_objective:
            # print("Improved")
            stable = 0
        else:
            stable += 1

        # Cập nhật lời giải kỉ lục
        if new_search_objective < getCost(bestSolve):
            bestSolve = np.array(solve)
        elif stable == stable_limit:
            # print("Jump")
            # solve = np.array(lastImproveSolve)
            stable = 0
            # Chuyển sang không gian khác
            solve = initialSolution(SEED+it)
            # Update tabu
            tabuList = np.zeros(N)

        # print("Gen{}: {} {}".format(it,getCost(solve),getCost(bestSolve)))
        # show(bestSolve)
        f.write("Gen{}: {} {}\n".format(it, getCost(solve), getCost(bestSolve)))
    f.close()
    return bestSolve

In [ ]:
def logOutput(outputPath, Name, time, SEED, sol):
    f = open(outputPath, "w")
    f.write("Name: "+Name+"\n")
    f.write("Time: {} \n".format(time))
    f.write("Result: {}\n".format(getCost(sol)))
    f.write("Solution: ")
    for i in sol:
        f.write(" {}".format(i))
    f.write("\n")
    amount = np.zeros(date)
    day = [[] for i in range(date)]
    for i in range(N):
        amount[sol[i]] += d[i]
        day[sol[i]].append(i)

    for i in range(date):
        f.write("{} : {} ({})\n".format(i, day[i], int(amount[i])))
        # print("----sum: {}".format(amount[i]))

In [ ]:
for i in range(1, 5):
    os.mkdir("Output/Type"+str(i)+"Small")
    os.mkdir("Output/Type"+str(i)+"Big")

# Main


In [ ]:
# # Small
# m_list = [10, 20, 35, 50]
# M_list = [15, 30, 50, 75]
# N_list = [10, 20, 40, 60]

# Big
m_list = [20, 30, 40, 50]
M_list = [30, 45, 60, 80]
N_list = [200, 400, 600, 800]

# TypeData = ["Type1Small", "Type2Small", "Type3Small", "Type4Small"]
TypeData = ["Type1Big", "Type2Big", "Type3Big", "Type4Big"]

for Type in TypeData:
    for i in range(4):
        for j in range(4):
            Name = str(N_list[i])+"-"+str(m_list[j])+"-"+str(M_list[j])
            loadData(Type, Name)

            for SEED in range(10, 20):
                outputPath = "Output/"+Type+"/"+Name+"_SEED_"+str(SEED)
                print("Output/"+Type+"/"+Name+"_SEED_"+str(SEED)+".txt")
                start = time.time()
                solution = TabuSearch(outputPath + "_gen.txt", Name, SEED)
                run_time = time.time() - start
                logOutput(outputPath + ".txt", Name, run_time, SEED, solution)


Output/Type1Big/200-20-30_SEED_10.txt
Output/Type1Big/200-20-30_SEED_11.txt
Output/Type1Big/200-20-30_SEED_12.txt
Output/Type1Big/200-20-30_SEED_13.txt
Output/Type1Big/200-20-30_SEED_14.txt
Output/Type1Big/200-20-30_SEED_15.txt
Output/Type1Big/200-20-30_SEED_16.txt
Output/Type1Big/200-20-30_SEED_17.txt
Output/Type1Big/200-20-30_SEED_18.txt
Output/Type1Big/200-20-30_SEED_19.txt
Output/Type1Big/200-30-45_SEED_10.txt
Output/Type1Big/200-30-45_SEED_11.txt
Output/Type1Big/200-30-45_SEED_12.txt
Output/Type1Big/200-30-45_SEED_13.txt
Output/Type1Big/200-30-45_SEED_14.txt
Output/Type1Big/200-30-45_SEED_15.txt
Output/Type1Big/200-30-45_SEED_16.txt
Output/Type1Big/200-30-45_SEED_17.txt
Output/Type1Big/200-30-45_SEED_18.txt
Output/Type1Big/200-30-45_SEED_19.txt
Output/Type1Big/200-40-60_SEED_10.txt
Output/Type1Big/200-40-60_SEED_11.txt
Output/Type1Big/200-40-60_SEED_12.txt
Output/Type1Big/200-40-60_SEED_13.txt
Output/Type1Big/200-40-60_SEED_14.txt
Output/Type1Big/200-40-60_SEED_15.txt
Output/Type1